In [11]:
import pandas as pd
import numpy as np
import os
import pandas as pd
import warnings
import sys
from transformers import AutoModelForSequenceClassification, AutoTokenizer


pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

model_name = "roberta-large-mnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
for name, param in model.named_parameters():
    print(name, param.size())

roberta.embeddings.word_embeddings.weight torch.Size([50265, 1024])
roberta.embeddings.position_embeddings.weight torch.Size([514, 1024])
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 1024])
roberta.embeddings.LayerNorm.weight torch.Size([1024])
roberta.embeddings.LayerNorm.bias torch.Size([1024])
roberta.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
roberta.encoder.layer.0.attention.self.query.bias torch.Size([1024])
roberta.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024])
roberta.encoder.layer.0.attention.self.key.bias torch.Size([1024])
roberta.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024])
roberta.encoder.layer.0.attention.self.value.bias torch.Size([1024])
roberta.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
roberta.encoder.layer.0.attention.output.dense.bias torch.Size([1024])
roberta.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([1024])
roberta.encoder.layer.0

In [120]:
from transformers import pipeline
import torch
from datasets import load_dataset

ds = load_dataset("wangrongsheng/ag_news")


# if torch.cuda.is_available():
#     device = torch.device("cuda")
# elif torch.backends.mps.is_built():
#     device = torch.device("mps")
# else:
device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cpu


In [ ]:
print(ds["test"])
print(ds["test"][0])
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli', device=0)                              
sequence_to_classify = "badhiyab hai badhiya"
candidate_labels = ['travel', 'cooking', 'dancing', 'cricket', 'sports person', 'footballer', 'soccer player', 'actor']
classifier(sequence_to_classify, candidate_labels)


Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})
{'text': "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.", 'label': 2}


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'sequence': 'badhiyab hai badhiya',
 'labels': ['cooking',
  'dancing',
  'travel',
  'sports person',
  'actor',
  'footballer',
  'soccer player',
  'cricket'],
 'scores': [0.2231716364622116,
  0.16547532379627228,
  0.1618598997592926,
  0.13351215422153473,
  0.10129763185977936,
  0.07446768134832382,
  0.07138216495513916,
  0.06883353739976883]}

In [138]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict

def perform_error_analysis(dataset, classifier, num_samples=100):
    """
    Perform comprehensive error analysis on the zero-shot classifier.
    
    Args:
        dataset: The AG News dataset
        classifier: The zero-shot classification pipeline
        num_samples: Number of samples to analyze (use smaller number for testing)
    """
    # AG News labels mapping
    label_map = {
        0: "This document is about politics",
        1: "This document is about sports",
        2: "This document is about economics",
        3: "This document is about science and technology"
    }
    
    # Prepare candidate labels for zero-shot classification
    candidate_labels = list(label_map.values())
    
    # Store results
    results = []
    
    # Process test samples
    for i, item in tqdm(enumerate(dataset['test']), total=num_samples):
        if i >= num_samples:
            break
            
        text = item['text']
        true_label = label_map[item['label']]
        
        # Get model prediction
        prediction = classifier(text, candidate_labels)
        predicted_label = prediction['labels'][0]
        confidence = prediction['scores'][0]
        
        
        
        results.append({
            'text': text,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'confidence': confidence,
            'correct': true_label == predicted_label
        })
    
    # Convert to DataFrame
    df_results = pd.DataFrame(results)
    
    # 1. Overall Accuracy
    accuracy = (df_results['correct'].sum() / len(df_results)) * 100
    print(f"\nOverall Accuracy: {accuracy:.2f}%")
    
    # 2. Per-class Performance
    print("\nPer-class Performance:")
    print(classification_report(df_results['true_label'], df_results['predicted_label']))
    
    # 3. Confusion Matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(df_results['true_label'], df_results['predicted_label'], labels=candidate_labels)
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=candidate_labels, yticklabels=candidate_labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 4. Confidence Analysis
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='true_label', y='confidence', hue='correct', data=df_results)
    plt.title('Confidence Distribution by Class and Correctness')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 5. Error Examples Analysis
    print("\nMost Confident Mistakes:")
    mistakes = df_results[~df_results['correct']].sort_values('confidence', ascending=False)
    print(mistakes[['text', 'true_label', 'predicted_label', 'confidence']].head())
    
    # 6. Length Analysis
    df_results['text_length'] = df_results['text'].str.len()
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='true_label', y='text_length', hue='correct', data=df_results)
    plt.title('Text Length Distribution by Class and Correctness')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    return df_results

In [139]:
def analyze_error_patterns(df_results):
    """
    Analyze specific patterns in the errors.
    """
    # Common misclassification patterns
    error_patterns = defaultdict(int)
    for _, row in df_results[~df_results['correct']].iterrows():
        pattern = f"{row['true_label']} → {row['predicted_label']}"
        error_patterns[pattern] += 1
    
    print("\nCommon Error Patterns:")
    for pattern, count in sorted(error_patterns.items(), key=lambda x: x[1], reverse=True):
        print(f"{pattern}: {count}")
        
    # Confidence threshold analysis
    thresholds = np.arange(0.1, 1.0, 0.1)
    accuracies = []
    coverage = []
    
    for threshold in thresholds:
        filtered_preds = df_results[df_results['confidence'] >= threshold]
        if len(filtered_preds) > 0:
            acc = (filtered_preds['correct'].sum() / len(filtered_preds)) * 100
            cov = (len(filtered_preds) / len(df_results)) * 100
            accuracies.append(acc)
            coverage.append(cov)
    
    # Ensure arrays are the same length before plotting
    min_len = min(len(accuracies), len(coverage))
    thresholds = thresholds[:min_len]
    accuracies = accuracies[:min_len]
    coverage = coverage[:min_len]
    
    if min_len > 0:  # Only plot if we have data
        plt.figure(figsize=(10, 6))
        plt.plot(thresholds, accuracies, 'b-', label='Accuracy')
        plt.plot(thresholds, coverage, 'r-', label='Coverage')
        plt.xlabel('Confidence Threshold')
        plt.ylabel('Percentage')
        plt.title('Accuracy vs Coverage Trade-off')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print("Warning: Not enough data points to create accuracy-coverage plot")


In [141]:
# Example usage
if __name__ == "__main__":
    # Perform main error analysis
    results_df = perform_error_analysis(ds, classifier, num_samples=500)
    
    # Analyze error patterns
    analyze_error_patterns(results_df)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [45]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="Qwen/Qwen2.5-32B-Instruct")
pipe(messages)

model-00006-of-00017.safetensors:  39%|###9      | 1.53G/3.90G [00:00<?, ?B/s]

model-00006-of-00017.safetensors:  40%|###9      | 1.55G/3.90G [00:00<?, ?B/s]

model-00007-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00008-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00009-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/e3/79/e3796211fc025913b53a6b38d22e1cbcc7bb19c33761017c256532b8d0e1bc3d/47f562bf77d986b53c5390b01a30a77576c3eed69191b484cf88b53bb64bce19?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00009-of-00017.safetensors%3B+filename%3D%22model-00009-of-00017.safetensors%22%3B&Expires=1731932779&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTkzMjc3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UzLzc5L2UzNzk2MjExZmMwMjU5MTNiNTNhNmIzOGQyMmUxY2JjYzdiYjE5YzMzNzYxMDE3YzI1NjUzMmI4ZDBlMWJjM2QvNDdmNTYyYmY3N2Q5ODZiNTNjNTM5MGIwMWEzMGE3NzU3NmMzZWVkNjkxOTFiNDg0Y2Y4OGI1M2JiNjRiY2UxOT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=A8pJGohbCVm9-NFbyy7-MTdcW4xBd3nQvGFd7KH1k5ouHfWnP8XoBG6WrHDTDkrYt8G6h0-hxUzjL2g7K4UpOtWHc2qwyWlBgkXJHx-Qp10UuPXbh0-WuNSkzpwK8FgyppAyDjpss5OXo5ThrnJIE4vER7SLf000RVIMbyjIPcLl8ctLRmb4%7EFW9yQJ%7EyY97%7EJYGjohmrlrEEF8262XmmsD9Uz8VfWgk

ValueError: Could not load model Qwen/Qwen2.5-32B-Instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/root/anaconda3/envs/nst/lib/python3.10/site-packages/urllib3/response.py", line 748, in _error_catcher
    yield
  File "/root/anaconda3/envs/nst/lib/python3.10/site-packages/urllib3/response.py", line 894, in _raw_read
    raise IncompleteRead(self._fp_bytes_read, self.length_remaining)
urllib3.exceptions.IncompleteRead: IncompleteRead(27048283 bytes read, 2342878301 more expected)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/root/anaconda3/envs/nst/lib/python3.10/site-packages/requests/models.py", line 820, in generate
    yield from self.raw.stream(chunk